<a href="https://colab.research.google.com/github/KanoknatKr/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

#1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)

## 1.1 เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก google drive โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้ (5 คะแนน)
'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 'sub_district', 'zone' จากไฟล์ "Data House" - บ้านมือ 2

In [ ]:
import pandas as pd
import json
from math import radians, sin, cos, sqrt, atan2
from google.colab import drive
drive.mount('/content/drive')

# Path files
data_house_path = '/content/drive/My Drive/bsc_dpdm24_data/Data_House.csv'
seven_path = '/content/drive/My Drive/bsc_dpdm24_data/7-eleven.json'
school_path = '/content/drive/My Drive/bsc_dpdm24_data/school.json'
appraisal_path = '/content/drive/My Drive/bsc_dpdm24_data/คำตอบนักประเมิน(ID).csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1.1 Load the data from the CSV file
data_house = pd.read_csv(data_house_path)

# Select the columns as per the requirement, including 'lat' and 'long'
data_house_selected = data_house[['renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va',
                                  'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
                                  'direction', 'furniture', 'usable_area', 'number_of_parking',
                                  'sell_price', 'district', 'sub_district', 'zone',
                                  'latitude', 'longitude']]  # Added 'lat' and 'long' here

# Check the first few rows to make sure the data looks correct
data_house_selected

,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone,latitude,longitude
0,no,0,1,45.0,2.0,2.0,2,ไม่ระบุ,ไม่มี,244.00,2.0,390000.0,ชนบท,ศรีบุญเรือง,NaN,16.139321,102.631587
1,no,0,0,88.0,1.0,1.0,1,ไม่ระบุ,ไม่มี,164.00,3.0,820000.0,น้ำพอง,ทรายมูล,NaN,16.676156,102.917998
2,no,1,0,8.0,1.0,11.0,11,ไม่ระบุ,ไม่ระบุ,489.00,9.0,3260000.0,มัญจาคีรี,กุดเค้า,NaN,16.129358,102.530670
3,no,0,0,61.6,2.0,3.0,3,ไม่ระบุ,ไม่ระบุ,203.00,2.0,5000000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน,16.459052,102.806404
4,no,0,0,72.3,2.0,3.0,2,ไม่ระบุ,ไม่ระบุ,164.00,2.0,3500000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน,16.450000,102.800680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,no,0,0,73.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,108.76,NaN,437400.0,บ้านฝาง,หนองบัว,NaN,16.480110,102.574033
1652,no,0,0,84.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,119.00,NaN,714600.0,หนองเรือ,โนนทอง,NaN,16.528551,102.411762
1653,no,0,1,45.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,188.50,NaN,451000.0,แวงน้อย,ก้านเหลือง,NaN,15.867444,102.482085
1654,no,0,1,59.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,207.00,NaN,910000.0,ชุมแพ,วังหินลาด,NaN,16.636912,102.105343


## 1.2 จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school') จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ 7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

In [ ]:
# prompt: จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school') จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ 7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร)

import pandas as pd
import json
from math import radians, sin, cos, sqrt, atan2

# Load 7-eleven data
with open(seven_path, 'r') as f:
    seven_data = json.load(f)

# Load school data
with open(school_path, 'r') as f:
    school_data = json.load(f)

def distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Create lists to store the counts
d1000_711 = []
d1000_school = []

for index, row in data_house_selected.iterrows():
    house_lat = row['latitude']
    house_lon = row['longitude']

    seven_count = 0
    for seven in seven_data:
        # Check if 'seven' is a dictionary before accessing 'latitude' and 'longitude'
        if isinstance(seven, dict) and 'latitude' in seven and 'longitude' in seven:
            if distance(house_lat, house_lon, seven['latitude'], seven['longitude']) <= 1:
                seven_count += 1
    d1000_711.append(seven_count)

    school_count = 0
    for school in school_data:
        # Check if 'school' is a dictionary before accessing 'latitude' and 'longitude'
        if isinstance(school, dict) and 'latitude' in school and 'longitude' in school:
            if distance(house_lat, house_lon, school['latitude'], school['longitude']) <= 1:
                school_count += 1
    d1000_school.append(school_count)

# Add the new columns to the DataFrame
data_house_selected['d1000-7/11'] = d1000_711
data_house_selected['d1000-school'] = d1000_school

<ipython-input-60-b449db58a057>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_house_selected['d1000-7/11'] = d1000_711
<ipython-input-60-b449db58a057>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_house_selected['d1000-school'] = d1000_school


In [ ]:
data_house_selected

,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone,latitude,longitude,d1000-7/11,d1000-school
0,no,0,1,45.0,2.0,2.0,2,ไม่ระบุ,ไม่มี,244.00,2.0,390000.0,ชนบท,ศรีบุญเรือง,NaN,16.139321,102.631587,0,0
1,no,0,0,88.0,1.0,1.0,1,ไม่ระบุ,ไม่มี,164.00,3.0,820000.0,น้ำพอง,ทรายมูล,NaN,16.676156,102.917998,0,0
2,no,1,0,8.0,1.0,11.0,11,ไม่ระบุ,ไม่ระบุ,489.00,9.0,3260000.0,มัญจาคีรี,กุดเค้า,NaN,16.129358,102.530670,0,0
3,no,0,0,61.6,2.0,3.0,3,ไม่ระบุ,ไม่ระบุ,203.00,2.0,5000000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน,16.459052,102.806404,0,0
4,no,0,0,72.3,2.0,3.0,2,ไม่ระบุ,ไม่ระบุ,164.00,2.0,3500000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน,16.450000,102.800680,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,no,0,0,73.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,108.76,NaN,437400.0,บ้านฝาง,หนองบัว,NaN,16.480110,102.574033,0,0
1652,no,0,0,84.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,119.00,NaN,714600.0,หนองเรือ,โนนทอง,NaN,16.528551,102.411762,0,0
1653,no,0,1,45.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,188.50,NaN,451000.0,แวงน้อย,ก้านเหลือง,NaN,15.867444,102.482085,0,0
1654,no,0,1,59.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,207.00,NaN,910000.0,ชุมแพ,วังหินลาด,NaN,16.636912,102.105343,0,0


## 1.3 ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score') จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

In [ ]:
# อ่านข้อมูลจากไฟล์ CSV
appraisal_data = pd.read_csv('/content/drive/My Drive/bsc_dpdm24_data/คำตอบนักประเมิน(ID).csv')

# ใช้ regular expression เพื่อดึงตัวอักษรแรกจากคอลัมน์ 'ให้คะแนนทรัพย์'
appraisal_data['ให้คะแนนทรัพย์'] = appraisal_data['ให้คะแนนทรัพย์'].str.extract(r'^[^\d]*(\d+)')

# แปลงค่าให้เป็นตัวเลข (float) ถ้าจำเป็น
appraisal_data['ให้คะแนนทรัพย์'] = appraisal_data['ให้คะแนนทรัพย์'].astype(float)

# ตรวจสอบข้อมูล
print(appraisal_data.head())

   ID ผู้ประเมิน                              post id   ID property_type  \
0              1   https://nayoo.co/khonkaen/posts/25   25    บ้านเดี่ยว   
1              2   https://nayoo.co/khonkaen/posts/25   25    บ้านเดี่ยว   
2              1   https://nayoo.co/khonkaen/posts/33   33    บ้านเดี่ยว   
3              2   https://nayoo.co/khonkaen/posts/33   33    บ้านเดี่ยว   
4              1  https://nayoo.co/khonkaen/posts/495  495    บ้านเดี่ยว   

  ความมั่นใจในการเมินราคาทรัพย์ ราคาทรัพย์ ประเมินราคาซื้อ-ขาย  \
0                        มั่นใจ  2,890,000           2,500,000   
1                        มั่นใจ  2,890,000           2,700,000   
2                        มั่นใจ  2,600,000           2,200,000   
3                        มั่นใจ  2,600,000           2,400,000   
4                        มั่นใจ  6,050,000           5,000,000   

                      ปัจจัยในการประเมินราคาซื้อ-ขาย  \
0                                       5,000-15,000   
1  โครงการนี้ราคาเสนอขาย 2,500,000

In [ ]:
# อ่านข้อมูลจากไฟล์ CSV
appraisal_data = pd.read_csv(appraisal_path)

# กรองข้อมูลเฉพาะนักประเมิน ID=01
appraisal_data_01 = appraisal_data[appraisal_data['ID'] == '01']

# สร้าง dictionary เพื่อเก็บราคาประเมินและคะแนนทรัพย์ โดยใช้ house_id เป็น key
appraisal_dict = {}
for index, row in appraisal_data_01.iterrows():
    house_id = row['house_id']  # คอลัมน์ที่เป็นตัวระบุบ้านใน appraisal_data
    estimator = row['ประเมินราคาซื้อ-ขาย']
    score = row['ให้คะแนนทรัพย์']
    appraisal_dict[house_id] = {'estimator': estimator, 'score': score}

# เพิ่มคอลัมน์ 'estimator' และ 'score' ใน data_house_selected
data_house_selected['estimator'] = 0
data_house_selected['score'] = 0

# เติมข้อมูลจาก appraisal_dict ลงใน data_house_selected
for index, row in data_house_selected.iterrows():
    house_id = row['id']  # ใช้ 'id' แทน 'house_id' ใน data_house_selected
    if house_id in appraisal_dict:
        data_house_selected.loc[index, 'estimator'] = appraisal_dict[house_id]['estimator']
        data_house_selected.loc[index, 'score'] = appraisal_dict[house_id]['score']

# แสดงข้อมูล
data_house_selected.head()

<ipython-input-63-d8549d98b08f>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_house_selected['estimator'] = 0
<ipython-input-63-d8549d98b08f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_house_selected['score'] = 0


KeyError: 'id'

In [ ]:
# ดูคอลัมน์ทั้งหมดใน data_house_selected
print(data_house_selected.columns)

Index(['renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va',
       'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
       'direction', 'furniture', 'usable_area', 'number_of_parking',
       'sell_price', 'district', 'sub_district', 'zone'],
      dtype='object')


#2. จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)

## 2.1 zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

In [ ]:
# prompt: zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

# Group data by 'zone' and calculate the mean 'sell_price' for each zone
zone_avg_prices = data_house_selected.groupby('zone')['sell_price'].mean()

# Find the zone with the highest average 'sell_price'
highest_avg_price_zone = zone_avg_prices.idxmax()

print(f"The zone with the highest average selling price is: {highest_avg_price_zone}")

The zone with the highest average selling price is: ในเมือง-รื่นรมย์-เทพารักษ์


## 2.2 บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)

In [ ]:
# prompt: บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่

# Find the house with the maximum number of 7-11s within a 1km radius
max_711_house = data_house_selected['d1000-7/11'].max()
house_with_max_711 = data_house_selected[data_house_selected['d1000-7/11'] == max_711_house]

# Print the price of the house with the most 7-11s
print(f"The price of the house with the most 7-11s within a 1km radius is: {house_with_max_711['sell_price'].iloc[0]}")

KeyError: 'd1000-7/11'

ตอบ 8,000,000 บาท

## 2.3 ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [ ]:
# prompt: ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'ให้คะแนนทรัพย์' จาก appraisal_data เฉลี่ยจากนักประเมินมากที่สุด

# Group data by 'sub_district' and calculate the mean 'score' for each sub_district
# Changed 'ให้คะแนนทรัพย์' to 'score' to reflect the column name change
subdistrict_avg_scores = data_house_selected.groupby('sub_district')['score'].mean()

# Find the sub_district with the highest average 'score'
highest_avg_score_subdistrict = subdistrict_avg_scores.idxmax()

print(f"The sub_district with the highest average score is: {highest_avg_score_subdistrict}")

KeyError: 'Column not found: score'

ตอบ score = 4